In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm

##### 1. Load function **get_Data_On_Page, get_Data_Detail_1Product** from file **Main_function.py**

In [2]:
from Main_function import get_Data_On_Page, get_Data_Detail_1Product

# URL Lazada: dienthoai, sort price high to low
path = 'chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

#### 2. Crawl data

##### Crawl features displayed on the outside of the page

In [3]:
#Empty list, to store data from all pages
all_data = []

#Nhận link cho 5 page: 6-10 và sau đó đưa vào vòng lặp for cùng với hàm get_Data_On_Page
for page in tqdm(range(6, 11), desc="Crawling pages"):
    main_link = f'https://www.lazada.vn/catalog/?page={page}&q=dien%20thoai&sort=pricedesc'
    driver.get(main_link)
    sleep(random.uniform(2,5))                             #Random sleep to mimic human behavior and avoid getting blocked

    dienthoai = get_Data_On_Page(driver)
    all_data.append(dienthoai)                             

Crawling pages:   0%|          | 0/5 [00:00<?, ?it/s]

Crawling pages: 100%|██████████| 5/5 [02:46<00:00, 33.22s/it]


In [4]:
#Combine into a DataFrame
data_05_10 = pd.concat(all_data, ignore_index=True)
data_05_10.index = np.arange(1, len(data_05_10)+1)
data_05_10

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location
1,dien thoai,Điện thoại OPPO FIND N3 (16GB/512GB) - Hàng ch...,https://www.lazada.vn/products/dien-thoai-oppo...,41.990.000 ₫,7% Off,11 Đã bán,None,Hồ Chí Minh
2,dien thoai,Điện thoại Samsung Galaxy Z Fold 5 - Bảo hành ...,https://www.lazada.vn/products/dien-thoai-sams...,40.990.000 ₫,None,None,None,Hồ Chí Minh
3,dien thoai,Điện thoại Samsung Galaxy S24 Ultra 12GB/1TB -...,https://www.lazada.vn/products/dien-thoai-sams...,37.590.001 ₫,16% Off,24 Đã bán,(2),Hồ Chí Minh
4,dien thoai,[GIÁ SỐC CHỈ 6.6] [TẶNG Buds2 Pro + Duo Charge...,https://www.lazada.vn/products/gia-soc-chi-66-...,31.989.000 ₫,6% Off,1.4K Đã bán,(324),Bắc Ninh
5,dien thoai,Điện thoại Samsung Galaxy S23 Ultra,https://www.lazada.vn/products/dien-thoai-sams...,31.990.000 ₫,None,203 Đã bán,(43),Hồ Chí Minh
...,...,...,...,...,...,...,...,...
196,dien thoai,Điện thoại siêu bền 8849 TANK MINI( chống nước...,https://www.lazada.vn/products/dien-thoai-sieu...,8.480.000 ₫,6% Off,None,None,Hồ Chí Minh
197,dien thoai,Điện Thoại Xiaomi Redmi Note 13 Pro 5G - Hàng ...,https://www.lazada.vn/products/dien-thoai-xiao...,8.410.000 ₫,11% Off,None,None,Hồ Chí Minh
198,dien thoai,[Nhập mã 66LAZSOCIAL80] Điện thoại HONOR X9b 5...,https://www.lazada.vn/products/nhap-ma-66lazso...,8.390.000 ₫,7% Off,7 Đã bán,(1),Long An
199,dien thoai,Điện thoại Samsung Galaxy M54 5G (8GB/256GB) -...,https://www.lazada.vn/products/dien-thoai-sams...,8.390.000 ₫,30% Off,14 Đã bán,None,Đà Nẵng


##### Crawl features data for each product

In [5]:
#Initialize dictionary to store details for all products
detail_products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(data_05_10.Link, desc = "Crawling product details"):
    details = get_Data_Detail_1Product(driver, link)
    for key in detail_products.keys():
        detail_products[key].append(details[key])

Crawling product details: 100%|██████████| 200/200 [1:40:10<00:00, 30.05s/it]


In [6]:
#Add columns to combined_data
for key, value in detail_products.items():
    data_05_10[key] = value

# Convert to csv
data_05_10.to_csv("Page_06_10.csv", index=False)
data_05_10

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Return,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
1,dien thoai,Điện thoại OPPO FIND N3 (16GB/512GB) - Hàng ch...,https://www.lazada.vn/products/dien-thoai-oppo...,41.990.000 ₫,7% Off,11 Đã bán,None,Hồ Chí Minh,44.990.000 ₫,20.100 ₫,30 Ngày Trả Hàng Miễn Phí,99%,100%,83%,[0],[0],[0],[0],[0]
2,dien thoai,Điện thoại Samsung Galaxy Z Fold 5 - Bảo hành ...,https://www.lazada.vn/products/dien-thoai-sams...,40.990.000 ₫,None,None,None,Hồ Chí Minh,None,18.300 ₫,None,97%,99%,88%,[0],[0],[0],[0],[0]
3,dien thoai,Điện thoại Samsung Galaxy S24 Ultra 12GB/1TB -...,https://www.lazada.vn/products/dien-thoai-sams...,37.590.001 ₫,16% Off,24 Đã bán,(2),Hồ Chí Minh,44.490.000 ₫,,30 Ngày Trả Hàng Miễn Phí,91%,Không đủ thông tin,100%,[0],[0],[0],[0],[2]
4,dien thoai,[GIÁ SỐC CHỈ 6.6] [TẶNG Buds2 Pro + Duo Charge...,https://www.lazada.vn/products/gia-soc-chi-66-...,31.989.000 ₫,6% Off,1.4K Đã bán,(324),Bắc Ninh,33.990.000 ₫,16.500 ₫,30 Ngày Trả Hàng Miễn Phí,91%,99%,100%,[2],[0],[1],[0],[320]
5,dien thoai,Điện thoại Samsung Galaxy S23 Ultra,https://www.lazada.vn/products/dien-thoai-sams...,31.990.000 ₫,None,203 Đã bán,(43),Hồ Chí Minh,None,,30 Ngày Trả Hàng Miễn Phí,99%,Không đủ thông tin,100%,[2],[0],[0],[0],[41]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,dien thoai,Điện thoại siêu bền 8849 TANK MINI( chống nước...,https://www.lazada.vn/products/dien-thoai-sieu...,8.480.000 ₫,6% Off,None,None,Hồ Chí Minh,8.990.000 ₫,Miễn phí,None,97%,100%,50%,[0],[0],[0],[0],[0]
197,dien thoai,Điện Thoại Xiaomi Redmi Note 13 Pro 5G - Hàng ...,https://www.lazada.vn/products/dien-thoai-xiao...,8.410.000 ₫,11% Off,None,None,Hồ Chí Minh,9.490.000 ₫,Miễn phí,None,100%,Không đủ thông tin,100%,[0],[0],[0],[0],[0]
198,dien thoai,[Nhập mã 66LAZSOCIAL80] Điện thoại HONOR X9b 5...,https://www.lazada.vn/products/nhap-ma-66lazso...,8.390.000 ₫,7% Off,7 Đã bán,(1),Long An,8.990.000 ₫,28.700 ₫,30 Ngày Trả Hàng Miễn Phí,99%,100%,94%,[0],[0],[0],[0],[2]
199,dien thoai,Điện thoại Samsung Galaxy M54 5G (8GB/256GB) -...,https://www.lazada.vn/products/dien-thoai-sams...,8.390.000 ₫,30% Off,14 Đã bán,None,Đà Nẵng,11.990.000 ₫,38.000 ₫,None,97%,82%,86%,[0],[0],[0],[0],[0]
